In [1]:
# Enseble classifier object. 

In [2]:
# Libraries

In [3]:
# general library osed for the fllowing functions and classes: 
import numpy as np
import pandas as pd 
import sklearn.preprocessing
import sklearn as sk 
import scipy as sy
import random
import math 

import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, LeaveOneOut, PredefinedSplit
# import method to create the confusion matrix: 
from sklearn.metrics import confusion_matrix

from nonconformist.nc import NcFactory
from nonconformist.icp import IcpClassifier

from sklearn.utils.estimator_checks import check_estimator

In [4]:
# class an functions
%run class_functions_2.ipynb

In [5]:
# STATIC VAR 
# class to set a static variable, for an unseen instance. 
class staticVar: 
    def __init__(self, instance, cols): 
            self.instance = instance 
            self.cols = cols
    def create_instance(self):        
        return pd.DataFrame(self.instance, columns=self.cols)


# ENSEMBLE CLASSIFIER
# Define Ensemble classifier Class and method 
class EnsembleClassifier:
    def __init__(self, ne, possible_classifiers, classifier_dist, feature_bag, discretization,
                missing_categorical, missing_numerical, k_opt, calibration, compute_proba):
        # -------------------------------------------------------------------------- 
        # TODO: 
        # here before of everything you should check if params passed are correct. 
        # --------------------------------------------------------------------------
        
        # --------------------------------------------------------------------------
        # --- DICTIONARIES and FEATURES ---
        
        # features: 
        self.features = 0
            
        # Create a set of estimators for ensemble classifier (an empty dictionary): 
        self.estimators_set = {} # since ne = 3 here i wznt 3 estimators (each estimator is (FTi, CLi))
        
        # Create dictionary to save the converters (convert element of df in integers)
        self.converter_set = {} # one for each feature transformer. 
        
        # --------------------------------------------------------------------------
        
        # set number of estimators (ne)
        self.ne = ne
        
        # set possible classifiers 
        self.possible_classifiers = possible_classifiers
        # set distribution of possible classifiers
        self.classifier_dist = classifier_dist
        
        # set feature bag 
        self.feature_bag = feature_bag
        # set the result of featire bag
        #self.featureBag_TF = False
        
        # set possible discretization
        self.discretization = discretization 
        # set the possible choice of discretization
        self.possible_discretizations = ['global', 'local', 'scaled', 'none']
        
        # set missing categorical
        self.missing_categorical = missing_categorical
        self.possible_missing_categorical = ['drop', 'most_frequent', 'random', 'random_uniform']

        # set missing numerical 
        self.missing_numerical = missing_numerical
        self.possible_missing_numerical = ['drop','avarage', 'random', 'random_uniform']
      
        # set k_opt
        self.k_opt = k_opt
        
        # set calibration
        self.calibration = calibration
        
        # set compute proba
        self.compute_proba = compute_proba
               
        
    def set_prob_params(self,x,y,continuity):
        # ---------------------------------------------------------------------------------------------------------------------
        # INPUTS: --> x = features of dataset, y = labels of dataset
        # ---------------------------------------------------------------------------------------------------------------------
        # Example: ne = 3 --> {(FT1,CL1), (FT2,CL2), (FT3,CL3)}
        # each FT needs the following parameters: --> featureBag, discretization, missing_categorical, missing_numerical.
        # Each SOC (self-opt-class) nedds following parameters: --> classifier, params_grid, k_opt, calibration, compute_proba.
        # ATTENTION for the classifiers --> the classifier and the params_grid, are given when the params grid are decided.
        # ---------------------------------------------------------------------------------------------------------------------
        
        # assign features to self.features:
        self.features = x
        
        # num_labels
        self.num_labels = len(np.unique(y))
        
        # Create a class to rebuild the entire dataset when you need: 
        sv = staticVar(self.features.values, self.features.columns)
        
        # loop over the number of estimators.
        for i in range(ne): 
            # do all random choice for every parameter: 
            # First of all, given the vector of probability for each classifier in classifiers, 
            # choose the classifier(with np.random choice(arry_of_class, size_of_out_arr, probabilities_of_each_entry) with respective param grid
            
            # START PARAMENTERS FOR CLASSIFIER: 
            
            # ---------------------------------------------------------------------------------------------------------------------
            # ------------- CLASSIFIER ---------------

            # array from which we have to coohse classifier: 
            all_poss_classifier_int = list(range(len(self.possible_classifiers)))
            # use array of probability for each classifier to choose the best. 
            choosen_classifier = np.random.choice(all_poss_classifier_int, size=1, p=self.classifier_dist)

            # take choosen classifier object from the list of classifiers and its relative parameters
            choosen_classifier_obj = self.possible_classifiers[choosen_classifier[0]][0]
            classifier_parameters = self.possible_classifiers[choosen_classifier[0]][1]
            #print(choosen_classifier_obj)
            #print(classifier_parameters)
            
            # ---------------------------------------------------------------------------------------------------------------------
            # ------------- K_OPT ---------------
            # Given in input a dictionary with values that are numbers in N and values that are probabilities between 0and 1,
            # choose randomly the k-opt for the leave-k-out validation. 
            k_vals_prob = list(self.k_opt.values())
            k_keys_ch = list(self.k_opt.keys())
            
            # choose randomly the value
            kval_ch_idx = np.random.choice(k_keys_ch, size=1, p=k_vals_prob)
            k_val_choosen = int(kval_ch_idx[0])
            #print(int(k_val_choosen))
                        
            # ---------------------------------------------------------------------------------------------------------------------
            # ------------- CALIBRATION ---------------
            # DO THE SAME THING DONE WITH K_OPT
            calibration_prob = list(self.calibration.values())
            calibration_ch = list(self.calibration.keys())
            
            # choose randomly the value
            calibration_idx = np.random.choice(calibration_ch, size=1, p=calibration_prob)
            calibration_choosen = (calibration_idx[0])
            if calibration_choosen != 'bagging': 
                # cast to float
                calibration_choosen = (calibration_choosen).astype(np.float)
            #print(calibration_choosen)
            #print(type(calibration_choosen))
                        
            # ---------------------------------------------------------------------------------------------------------------------
            # ------------- COMPUTE PROBA ---------------
            # DO THE SAME THING DONE WITH K_OPT
            compute_proba_prob = list(self.compute_proba.values())
            compute_proba_ch = list(self.compute_proba.keys())
            
            # choose randomly the value
            compute_proba_idx = np.random.choice(compute_proba_ch, size=1, p=compute_proba_prob)
            compute_proba_choosen = (compute_proba_idx[0])
            if compute_proba_choosen != 'bagging': 
                # cast to float
                compute_proba_choosen = float(compute_proba_choosen)
            #print(compute_proba_choosen)
                        
            # ---------------------------------------------------------------------------------------------------------------------                        
            
            # --- START PARAMENTERS FOR FEATURE TRANSFORMER ---: 
            
            # ---------------------------------------------------------------------------------------------------------------------
            # ------------- FEATURE BAG ---------------
            # Given in input the probability of having featurebag = True (so only one value),
            # Use np.random.choice to choose randomly feature bag. 

            # create list of possible choice: 
            choices = [True, False]
            # find proba --> True & proba --> False
            prob_true = self.feature_bag
            prob_false = 1-prob_true
            # Create list of probabilities: 
            probs = [prob_true,prob_false]
            # Choose True or false for feature bagging: 
            feature_bag_ch = bool(np.random.choice(choices, size=1, p=probs))
            #self.featureBag_TF = ch
            #print(feature_bag_ch)

            # ---------------------------------------------------------------------------------------------------------------------
            # ------------- DISCRETIZATION ---------------
            # Choose randomly the discretization for the feature transformer: 
            discret_choice = self.possible_discretizations
            probab_discret = self.discretization
            
            # choose ranfomly the discretization
            index_ch = np.random.choice(discret_choice, size=1, p=probab_discret)
            discret_ch = index_ch[0]
            print(discret_ch)            
            
            # ---------------------------------------------------------------------------------------------------------------------
            # ------------- Missing categorical ---------------
            # choose randomly missing categorical attribute 
            missing_cat_choice = self.possible_missing_categorical
            missing_cat_prob = self.missing_categorical
            
            # choose the categorical 
            index_ch_cat = np.random.choice(missing_cat_choice, size=1, p=missing_cat_prob)
            missing_cat_ch = index_ch_cat[0]
            #print(missing_cat_ch)    
            
            # ---------------------------------------------------------------------------------------------------------------------
            # ------------- Missing numerical ---------------
            # choose randomly missing numerical attribute
            missing_num_choice = self.possible_missing_numerical
            missing_num_prob = self.missing_numerical
            
            # choose numerical
            index_ch_num = np.random.choice(missing_num_choice, size=1, p=missing_num_prob)
            missing_num_ch = index_ch_num[0]
            #print(missing_num_ch)
            # ---------------------------------------------------------------------------------------------------------------------
            
            # --- Create object Classifier (CLi) and object Feature transformer (FTi) ---
            # Create Feature Trtansformer obj 
            FT = FeatureTransformer(feature_bag_ch, discret_ch, missing_cat_ch, missing_num_ch)
            # Create classifier Obj. 
            SOCL = selfOptimizingClassifier(choosen_classifier_obj, params_grid = classifier_parameters, k_opt = k_val_choosen, calibration = calibration_choosen, compute_proba = compute_proba_choosen)
            # ---------------------------------------------------------------------------------------------------------------------
            # --- TRAIN ON FEATURES AND LABELS ---
            # Use fitData(x,y) method to train self optimizing classifier.
            
            # ---------------------------------
            # --- FEATURE TANSFORMER PART ---            
            # fit FeatureTransformer FT, eith features and labels.  
            FT.fit(x,y,continuity)
            # preprocess data using transform method of FT
            x = FT.transform(x)
            #print(x)
            # ---------------------------------
            
            # Check if missing categorical and/or missing numerical are 'drop' and launch allignDrop()
            # function, for resize labels. 
            if(missing_cat_ch == 'drop' or missing_num_ch == 'drop'):
                # resize y (labels of your features)
                y = allignDrop(FT,y)
            
            # ---------------------------------
            # Convert dataset into integer
            # Create object converter: 
            converter = attToNumClass(x)
            # add your converter to the converter dictionary: 
            self.converter_set[i] = converter
            # ----------------------------------
            # convert your dataset: 
            x = converter.convert(x)
            
            
            # ---------------------------------- 
            # --- TRAIN CLASSIFIER PART ---
            # Prepare features and labels: 
            x_features = x.to_numpy()
            # labels are already numoy array: 
            y_labels = y             
            # Fit your classifier: 
            print("Training classifier")
            SOCL.fitData(x_features,y_labels)
             # ----------------------------------   
                
            print("Classifier trained !!!")
           
            # ---------------------------------------------------------------------------------------------------------------------
            # -- ADD PAIR OF FT AND CL TO TRAINED DICTIONARY, THE DICTIONARY OF ALL ESTIMATORS ---
            # Create pair of FT(i) and CL(i)
            pair_FT_CL =[FT, SOCL]    
            # Add to the dictionary of estimators. Use i as key to retrive easily each estimator
            self.estimators_set[i] = pair_FT_CL
            
            # Rebuild the instance of features. Foundamental to refresh your modified dataset. 
            x = sv.create_instance()
            self.features = sv.create_instance()
            print("------------------------------------------------")
            
        #return self to update     
        return self 
    
    
    
    # ---------------------------------------------------------------------------------------------------------------------
    # --- DEFINE PREDICT METHOD ---
    # INPUTS: unseen_x --> uneen instance, voting_type --> how to choose the class.
    # unseen_x --> if it enter as data frame you can convert it, using the 
    # voting_type --> check different 
    # ---------------------------------------------------------------------------------------------------------------------
    def predict_class(self, unseen_x, voting_type): 
                
        
        # Check if the inserted voting type is one of the possible voting_type
        check_vote_list = ['majority', 'weighted', 'track_record', 'conformal_majority','conformal_weighted', 'conformal_track_record']
        
        if check_vote_list.count(voting_type) == 0: 
            return "--- Voting type inserted NOT CORRECT ---"
        
        
        
        # else run the prediction
        else: 
            
            # Create list of accurancy, prediction, predict_proba, conformal_prediction:            
            # - accurancy
            accurancy_list = []
            # - pred
            pred_list = [] 
            
            # confusion matrix list
            conf_matrix_list = []
            
            # - conformal_prediction
            conformal_pred_list = []
            
            # - pred_proba 
            pred_proba_list = []
                                    
            
            # Before the loop assign to a variable the instance, because feature transformer
            # transform directly your instace not creating a new one. So in a for loop could be a problem. 
            
            # Create staticVar object for unseen instance. 
            sv = staticVar(unseen_x.values, unseen_x.columns)
            
            # Create staticVar object for all the features dataset in case you need it. 
            sv_all_features = staticVar(self.features.values, self.features.columns)
            
            #assign to unseen instance
            new_instance = unseen_x
            
            # For security rebuild the dataset correctly: 
            self.features = sv_all_features.create_instance()
            
            #print("-------- ALL DATA BEFORE FEATURE TRANSFORMER: -------------")
            #print(self.features)
            #print("-----------------------------------------------------------")
            # for each estimator (eache pair of FT and CL) you have to predict the class of new instance.        
            # loop over. the dictionary of converter and dictionary of estimators           
            for i in self.estimators_set: 
                # take the converter 
                #conv = self.converter_set.get(i) # using get method to retrive from dictinoray
                                
                # take the featureTransformer 
                FT_transf = self.estimators_set[i][0] # not using get method
                
                # take the classifier ith classifier: 
                s_opt_class = self.estimators_set[i][1] # not using get method 
                
                # ---------------------------------------------------
                
                # Preprocess data using FeatureTransformer FT (ith)
                print("-------- ITERATION:", i, " -------------") # only to see at which iteration i am 
                unseen_x_transf = FT_transf.transform(new_instance)
                #print("################## ISTANZA TRASFORMATA: #######################")
                #print(unseen_x_transf)    
                
                # convert entire dataset 
                all_data = FT_transf.transform(self.features)
                print("--- All data: ---")
                print(all_data.head())
                # create converter
                converter = attToNumClass(all_data)
                converted_unseen_x = converter.convert(unseen_x_transf)
                #print("--- Converted instance: ---")
                #print(converted_unseen_x)
                
                # ---------------------------------------------------
                                            
                # compute prediction using SO-CL(ith), based also on the voting_type choosen
                
                # - PREDICT 
                pred = s_opt_class.predict(converted_unseen_x)
                #print("---------------------------------")
                #print("------ Predict: ", pred, " ------")
                # append to list of prediction. 
                pred_list.append(np.take(pred, 0))
                #print("---------------------------------")
                
                # - ACCURANCY 
                # Add to accurancy list. 
                accurancy_list.append(s_opt_class.acc_conf)
                #print(s_opt_class.acc_conf)
                
                # - CONFUSION MATRIX: 
                # Append to the confusion_matrix_list
                conf_matrix_list.append(s_opt_class.conf_matrix)
                #print(s_opt_class.acc_conf)
               
                # - PREDICT_PROBA 
                pred_prob = s_opt_class.predict_proba(converted_unseen_x)
                # add to the predict proba list. 
                pred_proba_list.append(pred_prob)                    
                
                # - CONFORMAL_PREDICTION
                # you need to cast to np.array(the instance) 
                # 0.1 is epsilon and correspond to the significance of the conformal prediction. 
                pred_conformal = s_opt_class.conformal_predict(np.array(converted_unseen_x),0.1)
                # add to list of conformal prediction. 
                conformal_pred_list.append(pred_conformal)
                
                # SOME PRINTS FOR DEBUGGING
                print("---------------------------------")
                print("------ Predict: ", pred," ------")
                print("------ Predict Proba: ", pred_prob,"------")
                print("------ Conformal prediction: ", pred_conformal,"------")
                print("------ Accurancy: ", s_opt_class.acc_conf," ------")
                print("--- Conf matrix: ---")
                print(s_opt_class.conf_matrix)                           
                print("---------------------------------")
                
                
                # ---------------------------------------------------
                # REASSIGN EVERYTHING 
                #print("################ ISTANZA RIASSEGNATA: #########################")
                new_instance = sv.create_instance() # rebuild the original new instance.
                self.features = sv_all_features.create_instance() # rebuild the original dataset of features.
                #print(new_instance)
                # ---------------------------------------------------

                
            # --- CHOOSE CORRECT VOTYNG TYPE ---  
            # Check the voting type choosen and create method for each one. 
            if (voting_type == 'majority'): 
                # take mode of the pred list and return the choosen 
                class_choosen = max(set(pred_list), key = pred_list.count) # here we take the most frequent variable. 
                #print(pred_list)
                
            elif (voting_type == 'weighted'):
                # It works only for estimator classifier with predict_proba != 'model'
                # take the maximum from list of accurancy 
                
                if s_opt_class.predict_proba == 'model': 
                    print("--- NOT FEASIBLE, PREDICT PROBA MUST BE != model ---")
                    return "--- NOT FEASIBLE, PREDICT PROBA MUST BE != model ---"
                
                # put 0 instead of None. 
                accurancy_list = [0 if v is None else v for v in accurancy_list]
            
                # take index of the max. 
                idx = accurancy_list.index(max(accurancy_list))  
                
                # take the class predicted from the same classifer: 
                class_choosen = pred_list[idx]
                
                
            elif (voting_type == 'track_record'):
                # take each confusion matrix. 
                # take prediction of classificator. 
                # the predicted value indicates the idex of the column of conf_matrix to take. 
                # Normalize each value of that column by the sum of values of that column. 
                # So from you predicted matrix you have to obtain a vector. (containing normalized values)
                            
                
                # create a new list for normalized columns given by the confusion matrices related to the predicted label. 
                cols_cm_list = []
                
                # loop over predictions and confusion_matrix list. 
                for i in range(len(pred_list)): 
                    # take prediction (will be index for the column of your confusion_matrix)
                    label_idx = pred_list[i]
                    
                    
                    # Case in label_idx which is nan    
                    if math.isnan(label_idx):
                        # in case it is nan take the most frequent and do the same as before.  
                        label_idx = max(set(pred_list), key = pred_list.count)
                        #print(label_idx)                       
                        # take column of confusion  related to the prediction 
                        col_cm = conf_matrix_list[i][:,label_idx]                
                        # create lmbda function to normalize the column
                        divide = lambda x: x/(sum(col_cm)) # divide by the sum of the col values.
                        # normalize the column  
                        col_cm_norm = divide(col_cm)
                        # append to the cols_cm_list
                        cols_cm_list.append(col_cm_norm)
                    
                    # Case in which label_idx is not nan 
                    else:
                        #print(label_idx)
                        # take column of confusion  related to the prediction 
                        col_cm = conf_matrix_list[i][:,label_idx]                
                        # create lmbda function to normalize the column
                        divide = lambda x: x/(sum(col_cm)) # divide by the sum of the col values.
                        # normalize the column  
                        col_cm_norm = divide(col_cm)
                        # append to the cols_cm_list
                        cols_cm_list.append(col_cm_norm)
                        
                
                # Once you have list of normalized columns compute sum over rovws of the vectors
                # finally take the index of the highest value. That index will be the label. 
                
                # number of rows: 
                n_row = cols_cm_list[0].shape[0]
                
                # create list of values
                values_track = []
                
                # loop:
                for i in range(n_row): 
                    val = 0
                    for vec in cols_cm_list:
                        val = val + vec[i]
                    # append to the list of val
                    values_track.append(val)
                 
                # Take the choosen label, corresponding to the class: 
                class_choosen = values_track.index(max(values_track))
                
            
            elif (voting_type == 'conformal_majority'): 
                # only when calibration != 0
                # vector of TRUE and FALSE converted into 0 and 1 (1 == TRUE, 0 == FALSE) 
                # put boolean vectors of conformal prediction into a list. 
                # see thes e vectors as columns. 
                # sum by rows. 
                # take index of the row with the higher sum. 
                
                # CHECK IF CALIBRATION IS DIFFERENT FROM 0.  
                if s_opt_class.calibration == 0: 
                    print("--- NOT FEASIBLE CALIBRATION MUST BE != 0 ---")
                    return "--- NOT FEASIBLE CALIBRATION MUST BE != 0 ---" 
                
                                                
                # convert boolean into int: 
                for i in range(len(conformal_pred_list)):
                    #print(conformal_pred_list[i])
                    # here i need to check if the conformal prediction has predict a nan. 
                    
                    # Since a nan is read as a float value; check if it is a float. 
                    if (type(conformal_pred_list[i]) == float): 
                        # Since parameter are not feasible. 
                        # Compute directly the majority. 
                        class_choosen = max(set(pred_list), key = pred_list.count) # here we take the most frequent variable. 
                        return class_choosen
                    
                    else: 
                        conformal_pred_list[i] = (conformal_pred_list[i]).astype(int)
                                
                # WHAT WE NEED:
                # NUMBER OF ONES IN EACH VECTOR (VEC) --> sum of the vec
                # DIVIDE THE 1 BY THE NUMER OF ONES, THIS REPRESENT THE NEW VALUE TO PUT INSTEAD OF 1. 
                # SUBSTITUE ONES WITH NEW VALUES 
                
                for vec in conformal_pred_list: 
                    # substitute with an np.aray
                    vec = np.array(vec)
                    # number of ones: 
                    if sum(vec).any() == 0: 
                        new_val = 0 # case in which prediction done is nan, and so the sum of values inside the list is zero. 
                    else: 
                        new_val = 1/sum(vec)
                        
                    # substitute each 1 with (1/tot_ones)
                    for v in vec: 
                        if (v == 1).any(): 
                            v = new_val
                        else: 
                            v = v 

                    #print("##### ECCOMI #####")
                    #print(vec)
                    #print("##### ------ #####")       

                
                # number of rows (equals to number of possible classes): 
                #n_row = conformal_pred_list[0].shape[0]
                
                # create list of values
                values_conf_maj = []
                
                # compute sum of vectors, since are numpy array. 
                val = 0
                for vec in conformal_pred_list:
                    val = val + vec
                # append to the list of val
                values_conf_maj.append(val)
                 
                # Take the choosen label, corresponding to the class: 
                class_choosen = values_conf_maj.index(max(values_conf_maj))                
                
            
            elif (voting_type == 'conformal_weighted'):
                # using probabilities (predict_proba) 
                # So multiply each value of the list by the predict_proba value.            
                                
                # CHECK IF CALIBRATION IS DIFFERENT FROM 0.  
                if s_opt_class.calibration == 0: 
                    print("--- NOT FEASIBLE CALIBRATION MUST BE != 0 ---")
                    return "--- NOT FEASIBLE CALIBRATION MUST BE != 0 ---" 
                                                                
                # convert boolean into int: 
                for i in range(len(conformal_pred_list)):
                    #print(conformal_pred_list[i])
                    # here i need to check if the conformal prediction has predict a nan. 
                    
                    # Since a nan is read as a float value; check if it is a float. 
                    if (type(conformal_pred_list[i]) == float):                       
                        # Compute majority that it is the easier 
                        class_choosen = max(set(pred_list), key = pred_list.count) # here we take the most frequent variable. 
                        return class_choosen
                    
                    else: 
                        conformal_pred_list[i] = (conformal_pred_list[i]).astype(int)
                                
                # WHAT WE NEED:
                # NUMBER OF ONES IN EACH VECTOR (VEC) --> sum of the vec
                # DIVIDE THE 1 BY THE NUMER OF ONES, THIS REPRESENT THE NEW VALUE TO PUT INSTEAD OF 1. 
                # SUBSTITUE ONES WITH NEW VALUES 
                
                for vec in conformal_pred_list: 
                    # substitute with an np.aray
                    vec = np.array(vec)
                    # number of ones: 
                    if sum(vec).any() == 0: 
                        new_val = 0 # case in which prediction done is nan, and so the sum of values inside the list is zero. 
                    else: 
                        new_val = 1/sum(vec)
                        
                    # substitute each 1 with (1/tot_ones)
                    for v in vec: 
                        if (v == 1).any(): 
                            v = new_val
                        else: 
                            v = v 

                    #print("##### ECCOMI #####")
                    #print(vec)
                    #print("##### ------ #####")       

                
                # number of rows (equals to the possible classes): 
                #print(conformal_pred_list)
                #print(conformal_pred_list[0].shape)
                #n_row = conformal_pred_list[0].shape[0]
                #print(n_row)
                
                # create list of values
                values_conf_weigh = []

                # pred_proba_list
                
                # loop:

                val = 0
                for s in range(len(conformal_pred_list)): 
                    # take vector of 0,1. 
                    vec = conformal_pred_list[s][0]
                    #print(vec)
                    # take vector of probabilities, from predict_proba. 
                    for i in range(len(vec)):
                        proba = pred_proba_list[0]
                        #print("----")
                        #print(proba)
                        proba = [item for sublist in proba for item in sublist]
                        #print(proba) 
                        #print("----")
                        # compute multiplication. 
                        vec[i] = vec[i]*proba[i]

                    # compute sum
                    val = val + vec
                        
                # append to the list of val
                values_conf_weigh.append(val)
                    
                # DEBUGGING PRINT 
                #print("----------")
                #print(values_conf_weigh) # should be 3 values. 
                #print("----------")
                
                # Take the choosen label, corresponding to the class: 
                class_choosen = values_conf_weigh.index(max(values_conf_weigh))          
                
            
            elif (voting_type == 'conformal_track_record'):
                # Use conformal prediction and also predict proba. 
                # We take only where conformal prediction is one and we then we take the relative predict proba value of the same index. 
                # Normalize values of the columns. (sum of predict proba values where conformal prediction is one) 
                # Once has been normalized, you have to sum by rows. 
                
                # CHECK IF CALIBRATION IS DIFFERENT FROM 0.  
                if s_opt_class.calibration == 0: 
                    print("--- NOT FEASIBLE CALIBRATION MUST BE != 0 ---")
                    return "--- NOT FEASIBLE CALIBRATION MUST BE != 0 ---" 
                
                                                
                # convert boolean into int: 
                for i in range(len(conformal_pred_list)):
                    #print(conformal_pred_list[i])
                    # here i need to check if the conformal prediction has predict a nan. 
                    
                    # Since a nan is read as a float value; check if it is a float. 
                    if (type(conformal_pred_list[i]) == float): 
                        # Since parameter are not feasible. 
                        # Compute directly the majority. 
                        class_choosen = max(set(pred_list), key = pred_list.count) # here we take the most frequent variable. 
                        return class_choosen
                    
                    else: 
                        # Convert true and false in 0 and ones.
                        conformal_pred_list[i] = (conformal_pred_list[i]).astype(int)
                val = 0
                for s in range(len(conformal_pred_list)): 
                    # take vector of 0,1. 
                    vec = conformal_pred_list[s][0]
                    #print('##############')
                    #print(vec)
                    # index of ones: 
                    idx_ones = [index for index, element in enumerate(vec) if element == 1]
                    # take pred proba current list. 
                    proba = pred_proba_list[0][0]
                    #print("--- Proba: ", proba, "---") 
                    proba_arr = np.array(proba)
                    proba_index = proba_arr[idx_ones]
                    #print(idx_ones)
                    proba_ones_list = list(proba_index)
                    norm_sum = sum(proba_ones_list)
                    
                    
                    norm_prob_ones_list = np.array(proba_ones_list)/norm_sum
                    norm_prob_ones_list=list(norm_prob_ones_list)
                    
                    for v in range(len(vec)): 
                        value = vec[v]
                        if (value == 1).any(): 
                            vec[v] = norm_prob_ones_list[v]
                        else: 
                            vec[v] = vec[v]
                            
                            
                    # predict_values = proba[idx_ones]
                    #print("-----")
                    #print(proba_ones_list)
                    #print(norm_sum)
                    #print(norm_prob_ones_list)
                    #print(vec)
                    #print("-----") 
                
                #print(conformal_pred_list)
                # create list of values
                values_conf_track = []

                # compute sum of vectors, since are numpy array. 
                val = 0
                for vect in conformal_pred_list:
                    val = val + vect
                # append to the list of val
                values_conf_track.append(val)
            
                class_choosen = values_conf_track.index(max(values_conf_track))

            # return the winning class by the voting type choosen
            return class_choosen
                               
                